In [3]:
import sys
sys.path.append('../../../')


import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as md
from tqdm import tqdm
from scipy import stats

from BayesNewton import bayesnewton
import objax

from sklearn.linear_model import LinearRegression